# Data cleaning and Feature Enginerring
In this file, we will deal with some of the abnormal data and missing value issues that we discover in the exploratory data analysis. We will also do some transformation on some variables to make them more meaningful. 

We also considered using some macroeconomic data (such as unemployment rate, the price of S&P 500, etc), as we though that a bad economy might motivate more people to commit fraud, but since the prediction did not improve much, we did not include them in our final model and they won't be discussed further here.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
train = pd.read_csv('../data/raw_train.csv')
test = pd.read_csv('../data/raw_test.csv')

In [ ]:
for df in [train, test]:
    # missing values
    df.fillna(-1, inplace=True)
    
    # ordinal features
    df['vehicle_category'] = df['vehicle_category'].map({'Compact': 0, 'Medium': 1, 'Large': 2})
    
    # binary features
    for feature in ['gender', 'living_status']:
        unique_values = df[feature].unique()
        df[feature] = df[feature].map({unique_values[0]: 0, unique_values[1]: 1})
        df.rename(columns={feature: feature + str(unique_values[0])}, inplace=True)

## fraud

We first remove the observations whose the target variable is equal to -1.

In [22]:
train = train[train['fraud'] != -1]

## age_of_driver

From Wikipedia, the oldest verified person ever was 122 and the current oldest living person is 115. We are going to treat those whose age is over 100 to as missing values.

In [24]:
train.loc[train.age_of_driver > 100, 'age_of_driver'] = np.nan
test.loc[test.age_of_driver > 100, 'age_of_driver'] = np.nan

## annual_income
There are some -1 values in annual_income. We will set them to NA.

In [25]:
train.loc[train.annual_income == -1, 'annual_income'] = np.nan
test.loc[test.annual_income == -1, 'annual_income'] = np.nan

## zip_code
We transformed the zip code data into latitude and longitude using the data from https://www.unitedstateszipcodes.org/zip-code-database/.

In [27]:
import csv
with open('../data/zip_code_database.csv', newline='') as csvfile:
    csv_reader = csv.DictReader(csvfile, delimiter=',')
    zip_to_lat = {}
    zip_to_lon = {}
    for zip_data in csv_reader:
        zip_to_lat[zip_data['zip']] = float(zip_data['latitude'])
        zip_to_lon[zip_data['zip']] = float(zip_data['longitude'])
        
zip_to_lat['0'] = -1
zip_to_lon['0'] = -1

In [28]:
for df in [train, test]:
    df['latitude'] = df['zip_code'].apply(lambda x: zip_to_lat[str(x)])
    df['longitude'] = df['zip_code'].apply(lambda x: zip_to_lon[str(x)])

## claim_date

Next, we break the claim dates into year, month, day, etc.

In [30]:
for df in [train, test]:
    df['claim_date'] = pd.to_datetime(df['claim_date'])
    df['claim_year'] = df['claim_date'].dt.year
    df['claim_month'] = df['claim_date'].dt.month
    df['claim_day'] = df['claim_date'].dt.day
    df['claim_weekofyear'] = df['claim_date'].dt.weekofyear
    df['claim_weekday'] = df['claim_date'].dt.weekday
    df['claim_weekend'] = (df['claim_weekday'] >= 5).astype(int)
    df['elapsed_time'] = (dt.date(2018, 1, 1) - df['claim_date'].dt.date).dt.days
    df.drop(columns=['claim_day_of_week'], inplace=True)

## rating_per_claim

We also create a new feature named rating per claim, which is a transformation of two existing features: 

$$\text{safty_rating} / (\text{past_num_of_claims} + 1).$$

Our rationale was that the safety rating and the past number of claims are both indicators of a person’s credibility but the credibility indicated by safety rating can be “consumed” by the past number of claims. For a person with a high safety rating, the chance of fraud is still high if he/she has a high number of past claims. The addition of one in the denominator was to avoid dividing by zero.

In [34]:
for df in [train, test]:
    df['rating_per_claim'] = df.apply(lambda col: col['safty_rating'] / (col['past_num_of_claims'] + 1), axis=1)

## Imputation of missing values
Now, we will do an imputation for the missing values. For simplicity, we will do a mean/mode imputation for the continuous/categorical variables.

In [32]:
for df in [train, test]:
    # mean imputation for continuous variables
    for var in ['age_of_driver', 'annual_income', 'claim_est_payout', 'age_of_vehicle', 'latitude', 'longitude']:
        var_mean = df.loc[:, var].mean()
        df[var].fillna(var_mean, inplace=True)
        
    # mode imputation for categorical variables
    for var in ['marital_status', 'witness_present_ind']:
        var_mode = df.loc[:, var].mode()
        df[var].fillna(var_mode, inplace=True)       

## Export cleaned data

In [35]:
train.to_csv('../data/train_clean.csv', header=True)
test.to_csv('../data/test_clean.csv', header=True)